<a href="https://colab.research.google.com/github/JohnnyPeng123/NLP-USYD/blob/master/Lab07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 07 Dependency parsing


## Dependency Parsing Exercise

Let's parse the following sentences! (Use your skill that you learned in the lecture)

*  I prefer the morning flight through Denver
*  eat sushi with tuna
*  eat sushi with chopsticks


Do this parsing exercise with your tutor and ***check with the answer (will be released with the exercise solution)***


**Dependency parsing process:** (Need more detailed parsing step? Please watch the lecture 7 recording!)

You can start the parsing with finding the predicate. For example, the head of token 'John' is token 'lives', because 'John' is the suject object of the predicate 'lives'. The head of 'live' is a fake token 'ROOT', we do this just for simplificity that every sentence has a same root node, which is the head of the entire structure

## Off-the-Shelf Tool (Spacy Parser)
Now let's use [spaCy](https://spacy.io/) to automatically parse sentences.
The spacy parser was developed based on the neural network based parser.

Parse the sentence ***'The woman showed a wellshaped smile in the dark.'***


In [0]:
import spacy
#PrettyTable is a Python library for generating simple ASCII tables.
from prettytable import PrettyTable

#load the spacy api with the pre-trained statistical models for English. English multi-task CNN trained on OntoNotes
nlp = spacy.load("en_core_web_sm")

#define the sentence
sentence = "The woman showed a wellshaped smile in the dark."
parse = nlp(sentence)


x = PrettyTable()
#define column names
x.field_names = ["TokenID", "Token", "HeadID", "Dependency"]

#spacy does not provide the fake ROOT so add a row for the fake Root
x.add_row([0,"ROOT",0,"-"])

#parsing reculsively
for token in parse:
  if token.dep_=="ROOT":
    x.add_row([token.i+1,token.text,"0",token.dep_])
  else:  
    x.add_row([token.i+1,token.text,token.head.i+1,token.dep_])
    


print("Parsing Result with Spacy API")
#printing the table
print(x)

Parsing Result with Spacy API
+---------+------------+--------+------------+
| TokenID |   Token    | HeadID | Dependency |
+---------+------------+--------+------------+
|    0    |    ROOT    |   0    |     -      |
|    1    |    The     |   2    |    det     |
|    2    |   woman    |   3    |   nsubj    |
|    3    |   showed   |   0    |    ROOT    |
|    4    |     a      |   6    |    det     |
|    5    | wellshaped |   6    |    amod    |
|    6    |   smile    |   3    |    dobj    |
|    7    |     in     |   3    |    prep    |
|    8    |    the     |   9    |    det     |
|    9    |    dark    |   7    |    pobj    |
|    10   |     .      |   3    |   punct    |
+---------+------------+--------+------------+


## Parsing Visualisation
Spacy provides its built-in visualisation suite, [displaCy](https://spacy.io/api/top-level#displacy)

In [0]:
#spaCy comes with a built-in visualization suite
from spacy import displacy

doc = nlp(sentence)
displacy.render(doc, style='dep', jupyter=True, options={'distance':90})


Unfortunately, the parsing result is ***NOT*** perfect. 
The result shows that *the head of the token '**dark**' is 'in' rather than 'showed'*.


### How can we evaluate the parsing result?

[Universal Dependencies](http://universaldependencies.org/) is such a grammatical annotation effort. Basiclly, a bunch of linguists sit there to manually label 'which head is this token modifying' or 'which syntactic role is this token playing with respect to which predicate'. We call their annotations ground truth, and evaluate outputs from different systems against this ground truth. The higher accuracy the system achieves, the better this system is.

## Parsing Evaluation (with Universal Dependencies)

The following code shows the function for evaluating the performance of spaCy parser with Universal Dependencies.


*   [CoNLL-U formatted by Universal Dependencies](http://universaldependencies.org/format.html)



In [0]:
#CoNLL-U Parser parses a CoNLL-U formatted string into a nested python dictionary. 
!pip install conllu

Let's make a function for evaluation the parsing result

In [0]:
import numpy as np
import conllu as conllu
from conllu import parse

#text: the text that you would like to text
#gold: the gold-standard annotation for the text
def evaluate_sentence_parsing(text, gold):
  #parse the text with spacy API
  parse = nlp(text)
  #setup the table for the parsing result table
  x = PrettyTable()
  x.field_names = ["TokenID", "Token", "HeadID", "Dependency"]
  x.add_row([0,"root",0,"-"])

  for token in parse:
    if token.dep_=="ROOT":
      x.add_row([token.i+1,token.text,"0","root"])
      token.dep_="root"
    else:  
      x.add_row([token.i+1,token.text,token.head.i+1,token.dep_])
 
  #printing the parsing result
  print("Parsing Result with Spacy API")
  print(x)
  
  #setup the table for the parsing result table
  y = PrettyTable()
  y.field_names = ["TokenID", "Token", "HeadID", "Dependency"]
  y.add_row([0,"root",0,"-"])

  #parse the gold-standard with conll-u parser 
  sentences = conllu.parse(gold)
  sentence = sentences[0]

  for token in sentence:
    if token['deprel']=="root":
      y.add_row([token['id'],token['form'],"0",token['deprel']])
    else:  
      y.add_row([token['id'],token['form'],token['head'],token['deprel']])
  print("Gold-Standard Annotation")
  print(y)

  #summarise the list of predicted head
  pred_head = [t.head.i+1 if i != t.head.i else 0 for i, t in enumerate(parse)]
  #summarise the list of gold head
  gold_head = [token['head'] for i, token in enumerate(sentence)]
  
  #summarise the list of predicted dependency relations
  pred_dep = [t.dep_ for i, t in enumerate(parse)]
  #summarise the list of gold dependency relations
  gold_dep = [token['deprel'] for i, token in enumerate(sentence)]
  
  
  
  print("\n\nParsing Result vs Gold - Head")
  print(pred_head)
  print(gold_head)

  print("\n\nParsing Result vs Gold - Dependency")
  print(pred_dep)
  print(gold_dep)
  
  #performance evaluation - Unlabeled Attachment Score (UAS)
  #Unlabeled Attachment Score (UAS): the percent of words that have the correct heads
  uas_accuracy = np.sum([1 if g == p else 0 for g, p in zip(gold_head, pred_head)]) / len(gold_head)
  
  
  print("\n\nUnlabeled Attachment Score (UAS)")
  return uas_accuracy  

With the above function, let's try to test and evaluate it.
There are two evaluation metrics:
1. ** Unlabeled Attachment Score (UAS)**: the percent of words that have the correct heads
2. **Labeled Attachment Score (LAS)**: the percent of words that have the correct heads and labels 



In [0]:
#testing sentence
text = "The woman showed a wellshaped smile in the dark."

#gold-standard of the sentence
gold = """
# text = The woman showed a wellshaped smile in the dark.
1	The	the	DET	DEF	Definite=Def|PronType=Art	2	det	_	_
2	woman	woman	NOUN	SG-NOM	Number=Sing	3	nsubj	_	_
3	showed	show	VERB	PAST	Mood=Ind|Tense=Past|VerbForm=Fin	0	root	_	_
4	a	a	DET	IND-SG	Definite=Ind|PronType=Art	6	det	_	_
5	wellshaped	wellshaped	ADJ	POS	Degree=Pos	6	amod	_	_
6	smile	smile	NOUN	SG-NOM	Case=Nom	3	obj	_	_
7	in	in	ADP	_	_	9	prep	_	_
8	the	the	DET	DEF	Definite=Def|PronType=Art	9	det	_	_
9	dark	dark	ADJ	POS	Degree=Pos	3	obl	_	SpaceAfter=No
10	.	.	PUNCT	Period	_	3	punct	_	_


"""

print(evaluate_sentence_parsing(text, gold))

Parsing Result with Spacy API
+---------+------------+--------+------------+
| TokenID |   Token    | HeadID | Dependency |
+---------+------------+--------+------------+
|    0    |    root    |   0    |     -      |
|    1    |    The     |   2    |    det     |
|    2    |   woman    |   3    |   nsubj    |
|    3    |   showed   |   0    |    root    |
|    4    |     a      |   6    |    det     |
|    5    | wellshaped |   6    |    amod    |
|    6    |   smile    |   3    |    dobj    |
|    7    |     in     |   3    |    prep    |
|    8    |    the     |   9    |    det     |
|    9    |    dark    |   7    |    pobj    |
|    10   |     .      |   3    |   punct    |
+---------+------------+--------+------------+
Gold-Standard Annotation
+---------+------------+--------+------------+
| TokenID |   Token    | HeadID | Dependency |
+---------+------------+--------+------------+
|    0    |    root    |   0    |     -      |
|    1    |    The     |   2    |    det     |
|    

###Summary
The result shows that UAS are 0.8. 

It means that there are 2 incorrectly predicted head out of 10. 
1.   The gold head of token 'in' is 'dark', whereas the prediction is 'smile'
2.   The gold head of token 'dark' is 'showed', whereas the prediction is 'in'.

**UAS** is the percent of words that have the correct heads

**LAS** is the percent of words that have the correct heads and labels.


---



## Transition-based Dependency Parser

Transition-based Dependency Parser is simple form of greedy discriminative dependency parser. It build only one tree, in one left-to-right seep over the input


The transition-based model consists of:
*   a ***buffer***, storing unprocessed text, for example, at the every beginning, it stores the input tokens
*   a ***stack***, storing elements under-processing, by default, it always has a ROOT node.
*   a list of ***depenedency relations***, you can also consider this as the parsing results. it is usually a list of tuples, each of which is a token and its head.

The ***arc-standard*** algorithm has three actions which can be applied to change the states of buffer, stack and list:

*   **LEFTARC**: assert a head-dependent relation between the word at the top of stack and the word directly beneath it; remove the lower word from the stack. Take the following figure as an example, this will create a (1 -> 2) relation, and 2 will be removed from the stack
*   **RIGHTARC**: assert a head-dependent relation between the second word on the stack and the word at the top; remove the word at the top of the stack; Still use the previous figure as example, (2 -> 1) relation will be added to the relation list, and 1 will be removed from the stack.
*   **SHIFT**: remove the word from the front of the input buffer and push it onto the stack. Consider the following buffer, the '3' will removed from the buffer 

####Lecture 7 - slide 48
![alt text](https://drive.google.com/uc?export=view&id=1S09VzsLhwwBJadtUmOyrb5_gYRQMsBWs)



In [0]:
class Parse(object):
    def __init__(self, sentence):
        self.sentence = sentence
        self.stack = ["<ROOT>"]
        self.buffer = sentence.split()
        self.relations = []

   #transition setup
    def parse_step(self, action):
        assert action in ["SHIFT", "LEFTARC", "RIGHTARC"]
        if action == "SHIFT":
            assert len(self.buffer) > 0
            token = self.buffer.pop(0)
            self.stack.append(token)
        else:
            assert len(self.stack) >= 2
            if action == "LEFTARC":
                relation = (self.stack[-1], self.stack[-2])
                self.relations.append(relation)
                self.stack.pop(-2)
            else:
                relation = (self.stack[-2], self.stack[-1])
                self.relations.append(relation)
                self.stack.pop(-1)

    def parse(self, actions):
        print("Let's start:")
        output_parse_state(self)
        print("*" * 50)
        for action in actions:
            self.parse_step(action)
            print("after action:", action)
            output_parse_state(self)
            print("*" * 50)


def output_parse_state(parse):
    print("Stack:", " ".join(parse.stack))
    print("Buffer:", " ".join(parse.buffer))
    print("Relations:")
    for relation in parse.relations:
        print("  %s -> %s" % (relation[0], relation[1]))

In [0]:
sentence = "Book me the morning flight"

actions = ["SHIFT", "SHIFT", "RIGHTARC", "SHIFT", "SHIFT", "SHIFT", "LEFTARC", 
           "LEFTARC", "RIGHTARC", "RIGHTARC"]
           
parse_obj = Parse(sentence)
parse_obj.parse(actions)

Let's start:
Stack: <ROOT>
Buffer: Book me the morning flight
Relations:
**************************************************
after action: SHIFT
Stack: <ROOT> Book
Buffer: me the morning flight
Relations:
**************************************************
after action: SHIFT
Stack: <ROOT> Book me
Buffer: the morning flight
Relations:
**************************************************
after action: RIGHTARC
Stack: <ROOT> Book
Buffer: the morning flight
Relations:
  Book -> me
**************************************************
after action: SHIFT
Stack: <ROOT> Book the
Buffer: morning flight
Relations:
  Book -> me
**************************************************
after action: SHIFT
Stack: <ROOT> Book the morning
Buffer: flight
Relations:
  Book -> me
**************************************************
after action: SHIFT
Stack: <ROOT> Book the morning flight
Buffer: 
Relations:
  Book -> me
**************************************************
after action: LEFTARC
Stack: <ROOT> Book the f

###Discussion
Where is this list of actions come from? It looks magic, but how can I know what actions to apply? A classifier is trained to predict what the next action is given the current state.
This machine learning classifier takes the state of buffer, stack, previous list of relations as input, and predict what the next action is. 

For more details, please read*** Lecture 7 from slide 65***

##Exercise
Evaluate the spaCy model using Universal Dependencies annotations (You should provide the parsing performance with a metric: UAS)

You are going to select the index from 10 sentences provided, and calculated the **UAS** by using spacy model, you should also **visualise the parsing graph** from the spacy model.





In [0]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1vktZbQ_yjvUqTyf0qBhkRsyvDnK0ueMe'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('en_lines-ud-test.conllu')  

# If you want to work on your own laptop, you can download the file here: 
# https://drive.google.com/open?id=1vktZbQ_yjvUqTyf0qBhkRsyvDnK0ueMe

In [0]:
!pip install conllu
from io import open
from conllu import parse_incr
import spacy
import numpy as np
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
data_file = open("en_lines-ud-test.conllu", "r", encoding="utf-8")

sent_list = []
gold_head_list = []
n = 0

for tokenlist in parse_incr(data_file):
    n += 1
    if n > 20:
        break

    sent = tokenlist.metadata['text']
    gold_head = [token['head'] for i, token in enumerate(tokenlist)]    

    sent_list.append(sent)
    gold_head_list.append(gold_head)

**Choose one of the sentences as the input of the following form (next cell)**

In [0]:
for sent in sent_list:
    print(sent)
#Choose one of the sentences as the input of the following form (next cell)    

If you plot multiple charts, you will see a drop area for multi-chart fields.
This type of drop area is not displayed for single charts.
Drop areas
Moving fields to the category and series areas
When you move a field to the series area, the unique items of data within the field are displayed as data series in the chart.
These series are represented by colored data markers, and their names appear in the chart legend.
When you move a field to the category area, the unique items of data are displayed as categories, or related groups of data.
Each category consists of one point from each data series.
Category labels usually appear across the x axis of the chart, although this can vary depending on the type of chart you are using.
Moving fields to the data area
Data fields provide the values to be summarized in the chart.
When you move a field to the data area, the values from the field are used as the data that is measured in the chart.
A filter field is similar to a page field in a Micros

### Form [Please Complete this Part]

In [0]:
#@title Input sentence, calcualte UAS and visualize graphs
#@markdown Please select a sentence from above. 
#@markdown Then calculate UAS by using spacy model, and visualize
#The input sentence will be selected from the above cell
sent = "Moving fields to the category and series areas"  #@param {type: "string"}
index_sent = sent_list.index(sent)

gold_head = gold_head_list[index_sent]
# Please complete this part


UAS: 0.6250
